# Perform GSEA using GSEAPY  

Following the potocol defined here: https://gseapy.readthedocs.io/en/latest/gseapy_tutorial.html#use-gsea-command-or-gsea


In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina' # mac
import pandas as pd
import gseapy as gp
import numpy as np
import matplotlib.pyplot as plt

In [3]:
gp.__version__

'0.9.15'

## Create gene lists

In [14]:
createGlists()

Alpha-state 1
2045
318
Alpha-state 2
665
1332
Alpha-state 3
54
375
Alpha-state 4
121
1251
Beta-state1 genes
1674
290
Beta-state2 genes
665
1128
Beta-state3 genes
158
1052


In [11]:
def createGlists():
    import csv 
    from openpyxl import load_workbook
    

    with open('../dat/1908/20190812_Arrojo_genesets_filtered.gmt','w') as f:
        wb = load_workbook(filename = '../dat/1908/Arrojo_e_Drigo_biorxiv.alpha_states.xlsx')
        tsv_writer = csv.writer(f, delimiter='\t')
        for s in ['Alpha-state '+str(i) for i in range(1,5)]:
            print(s)
            ws=wb[s]
            df=pd.DataFrame(ws.values)
            df2 = df[1:]
            df2.columns=df.loc[0].tolist()
            df2=df2.set_index('gene_name')
            cols=df2.columns
            x=df2[(df2[cols[0]]>1)&(df2[cols[1]]>3)].index.tolist()
            print(len(x))
            x.insert(0,s +' up')
            x.insert(0,s +' up')
            tsv_writer.writerow(x)
            x=df2[(df2[cols[0]]< -1)&(df2[cols[1]]< -3)].index.tolist()
            print(len(x))
            x.insert(0,s +' down')
            x.insert(0,s +' down')
            tsv_writer.writerow(x)
            
        wb = load_workbook(filename = '../dat/1908/Arrojo_e_Drigo_biorxiv.beta_states.xlsx')
        for s in ['Beta-state{0} genes'.format(str(i)) for i in range(1,4)]:
            print(s)
            ws=wb[s]
            df=pd.DataFrame(ws.values)
            df2 = df[1:]
            df2.columns=df.loc[0].tolist()
            df2=df2.set_index('gene_name')
            cols=df2.columns
            x=df2[(df2[cols[0]]>1)&(df2[cols[1]]>3)].index.tolist()
            print(len(x))
            x.insert(0,s +' up')
            x.insert(0,s +' up')
            tsv_writer.writerow(x)
            
            x=df2[(df2[cols[0]]< -1)&(df2[cols[1]]< -3)].index.tolist()
            print(len(x))
            x.insert(0,s +' down')
            x.insert(0,s +' down')
            tsv_writer.writerow(x)
            

## alpha

In [15]:
def run_GSEA_gene(celltype,nperm=1000,rseed=1000,
             rnkfile="../dat/figdata/fig2_prom_ttest_res.csv",
             glist='../dat/glists/gsea.gmt',col=1,
             rank_metrics='odds',zth=3,nTop=500,**kwargs):
    '''
    col=1 - Z, 0 - logFC
    '''

    gene_exp_alpha = pd.read_csv(rnkfile)[['celltype','gene',rank_metrics]]
            
    #gene_exp_alpha.head()
    rnk = gene_exp_alpha.loc[gene_exp_alpha["celltype"]==celltype].drop(columns='celltype').sort_values(by=rank_metrics)
    if(rank_metrics=='odds'):
        rnk['odds'] =np.log2(rnk['odds'])
        rnk=rnk.reset_index(drop=True)
        df=rnk.drop(columns='gene')        
        np.random.seed(seed=rseed)
        for i in  rnk.index[(rnk['odds']==np.inf).tolist()].tolist():
            rnk.iloc[i,1]= float(df[~df.isin([np.inf])].max(0)*(1+np.random.uniform()/100))

        for i in  rnk.index[(rnk['odds']==-np.inf).tolist()].tolist():
            rnk.iloc[i,1]= float(df[~df.isin([-np.inf])].min(0)*(1+np.random.uniform()/100))
        rnk['odds']=rnk['odds']/max(abs(rnk['odds']))
        
    print(rnk.shape)
    print(rnk.head(1))
    print(rnk.tail(1))
    print('zth:{0}'.format(zth))
    #createGlists()
    gs_res_a = gp.prerank(rnk=rnk, # or data='./P53_resampling_data.txt'
                     gene_sets=glist, # enrichr library names or gmt file
                     #set permutation_type to phenotype if samples >=15
                     permutation_num=nperm, # reduce number to speed up test
                     outdir=None,  # do not write output to disk
                     no_plot=True, # Skip plotting
                     #weighted_score_type=1,
                     #ascending=False,
                        seed=rseed,
                     min_size=1,
                     max_size=5000,
                     processes=8,**kwargs)
                     #format='png')
    return(gs_res_a)


In [16]:
gs_res_a=run_GSEA_gene(celltype='alpha',
                       nperm=6000,
                       rseed=1000,
                       glist='../dat/1908/20190812_Arrojo_genesets_filtered.gmt')

gs_res_b=run_GSEA_gene(celltype='beta',
                       nperm=4000,
                       rseed=2000,
                       glist='../dat/1908/20190812_Arrojo_genesets_filtered.gmt')

display(gs_res_a.res2d[gs_res_a.res2d.pval<0.01].sort_index())
gs_res_b.res2d[gs_res_b.res2d.pval<0.01].sort_index()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log2
  


(21715, 2)
     gene      odds
0  ZNF106 -0.994725
          gene      odds
21714  PLEKHS1  0.806376
zth:3


2019-09-03 15:13:33,775 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


(21825, 2)
   gene      odds
0  PDHX -0.993998
        gene      odds
21824  TIGIT  0.968675
zth:3


2019-09-03 15:15:29,528 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Beta-state2 genes down,-0.204801,-1794.441516,0.0,0.0,1128,1094,GMDS;TPM4;MOK;SLC7A8;MTCH1;MBTPS1;CACFD1;EPHX1...,PPP3CC;HLA-A;NT5C;NT5C3B;CSRP1;GABARAPL1;ISOC2...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,


In [7]:
gene_exp_alpha = pd.read_csv("../dat/figdata/fig2_prom_ttest_res.csv",index_col=1)
#gene_exp_alpha.head()
df = gene_exp_alpha.loc[gene_exp_alpha["celltype"]=="alpha"].sort_values(by='odds', ascending=False)["odds"]
df=np.log2(df)

# replace inf to max exclude inf
#df=df.replace(np.inf,df[~df.isin([np.inf])].max(0))
#df=df.replace(-np.inf,df[~df.isin([-np.inf])].min(0))
df.to_csv('../dat/figdata/res.genes.a.rnk',sep='\t')
rnk = pd.read_table("../dat/figdata/res.genes.a.rnk", header=None)

for i in  rnk.index[(rnk[1]==np.inf).tolist()].tolist():
    rnk.iloc[i,1]= df[~df.isin([np.inf])].max(0)*(1+np.random.uniform()/100)

for i in  rnk.index[(rnk[1]==-np.inf).tolist()].tolist():
    rnk.iloc[i,1]= df[~df.isin([-np.inf])].min(0)*(1+np.random.uniform()/100)

#rnk.set_index(0)
#rnk.head()

# run gsea
# enrichr libraries are supported by gsea module. Just provide the name

gs_res_a = gp.prerank(rnk=rnk, # or data='./P53_resampling_data.txt'
                 gene_sets='../dat/1908/20190812_Arrojo_genesets_filtered.gmt', # enrichr library names or gmt file
                 #set permutation_type to phenotype if samples >=15
                 permutation_num=6000, # reduce number to speed up test
                 outdir=None,  # do not write output to disk
                 no_plot=True, # Skip plotting
                 #weighted_score_type=1,
                 #ascending=False,
                    seed=1000,
                 min_size=1,
                 max_size=5000,
                 processes=6)
                 #format='png')
    
gs_res_a.res2d.sort_index()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log2
  after removing the cwd from sys.path.
2019-09-03 15:00:57,009 No gene sets passed through filtering condition!!!, try new parameters again!
Note: check gene name, gmt file format, or filtering size.


SystemExit: 0

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## beta

In [24]:
gene_exp_alpha = pd.read_csv("../dat/figdata/fig2_prom_ttest_res.csv",index_col=1)
#gene_exp_alpha.head()
df = gene_exp_alpha.loc[gene_exp_alpha["celltype"]=="beta"].sort_values(by='odds', ascending=False)["odds"]
df=np.log2(df)

# replace inf to max exclude inf
#df=df.replace(np.inf,df[~df.isin([np.inf])].max(0))
df=df.replace(-np.inf,df[~df.isin([-np.inf])].min(0))
df.to_csv('../dat/figdata/res.genes.b.rnk',sep='\t')
rnk = pd.read_table("../dat/figdata/res.genes.b.rnk", header=None)

for i in  rnk.index[(rnk[1]==np.inf).tolist()].tolist():
    rnk.iloc[i,1]= df[~df.isin([np.inf])].max(0)*(1+np.random.uniform()/100)

for i in  rnk.index[(rnk[1]==-np.inf).tolist()].tolist():
    rnk.iloc[i,1]= df[~df.isin([-np.inf])].min(0)*(1+np.random.uniform()/100)

#rnk.set_index(0)
#rnk.head()


# run gsea
# enrichr libraries are supported by gsea module. Just provide the name

gs_res = gp.prerank(rnk=rnk, # or data='./P53_resampling_data.txt'
                 gene_sets='../dat/1908/20190812_Arrojo_genesets_filtered.gmt', # enrichr library names or gmt file
                 #set permutation_type to phenotype if samples >=15
                 permutation_num=4000, # reduce number to speed up test
                 outdir=None,  # do not write output to disk
                 no_plot=True, # Skip plotting
                 weighted_score_type=1,
                 #ascending=False,
                    seed=2000,
                 min_size=10,
                 max_size=5000,
                 processes=8)
                 #format='png')
    
gs_res.res2d.sort_index()

,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha-state 1,-0.185698,NaN,NaN,1.000000,2045,1986,TRIM41;ATP2A3;NCOR2;KMT2D;PRPF40B;NPEPL1;INS;S...,TMEM259;ATG9A;ETNK2;GIPC1;SLC25A39;PRR24;TMEM1...
Alpha-state 2,0.212311,0.732689,0.993000,1.000000,665,642,EDIL3;ARID1B;CCNYL1;PPP2R5C;TLE3;SLC39A11;SENP...,EDIL3;ARID1B;CCNYL1;PPP2R5C;TLE3;SLC39A11;SENP...
Alpha-state 3,0.308090,0.913401,0.611565,1.000000,54,51,GSTZ1;ANGPTL4;AGT;G6PC2;PEMT;CDO1;SIL1;LOXL4;C...,GSTZ1;ANGPTL4;AGT;G6PC2;PEMT;CDO1;SIL1;LOXL4;C...
Alpha-state 4,-0.234361,-1.079642,0.243902,0.273191,121,112,FABP5;ANXA2;TUBA1C;TDRD9;C1orf168;CDHR3;PDE4D;...,SERTAD1;RPL9;ERRFI1;NR4A3;YWHAH;WEE1;HBEGF;HSP...
Alpha-state 5,0.212683,0.737763,0.999250,1.000000,1163,1133,CKAP5;FTO;TVP23B;KPNB1;ATF7IP;MPZL1;SUGP2;FABP...,CKAP5;FTO;TVP23B;KPNB1;ATF7IP;MPZL1;SUGP2;FABP...
Alpha-state 6,0.259325,0.893709,0.839750,1.000000,637,610,MAP3K2;KCNK1;FAM214A;RUFY3;ELL;NINJ1;BTG3;DTNA...,MAP3K2;KCNK1;FAM214A;RUFY3;ELL;NINJ1;BTG3;DTNA...
Beta-state1 genes,0.191234,0.664415,1.000000,0.985546,1674,1633,TRIM41;ATP2A3;NCOR2;KMT2D;PRCC;NPEPL1;INS;SPAT...,TRIM41;ATP2A3;NCOR2;KMT2D;PRCC;NPEPL1;INS;SPAT...
Beta-state2 genes,0.218411,0.752953,0.990000,1.000000,665,644,CKAP5;ARID1B;MAP3K2;PPP2R5C;TLE3;FAM193A;CEP35...,CKAP5;ARID1B;MAP3K2;PPP2R5C;TLE3;FAM193A;CEP35...
Beta-state3 genes,0.207192,0.682976,0.964332,1.000000,158,148,RFX3;BTG3;ASB9;PLCH2;KRT8;SGMS2;MOK;SPAG1;DLK1...,RFX3;BTG3;ASB9;PLCH2;KRT8;SGMS2;MOK;SPAG1;DLK1...


### Save results

In [15]:
gs_res.res2d[['es','nes','pval','fdr','geneset_size','matched_size']].to_csv('../dat/figdata/GSEA_beta_arrogo_f_seed2000.csv')
gs_res_a.res2d[['es','nes','pval','fdr','geneset_size','matched_size']].to_csv('../dat/figdata/GSEA_alpha_arrogo_f_seed2000.csv')

from gseapy.plot import gseaplot, heatmap
terms = gs_res.res2d.index 
for i in range(len(terms)):
    gseaplot(gs_res.ranking, term=terms[i], **gs_res.results[terms[i]],ofname=terms[i]+'_b_seed2000_filtered.pdf')
terms = gs_res_a.res2d.index 
for i in range(len(terms)):
    gseaplot(gs_res_a.ranking, term=terms[i], **gs_res_a.results[terms[i]],ofname=terms[i]+'_a_seed2000_filtered.pdf')